In [2]:
import pandas as pd
import ollama
import re
import time
from tqdm import tqdm

# Codebook for climate contrarian claims
CODEBOOK = """
1.1: Ice/permafrost/snow cover isn't melting
1.2: We're heading into an ice age/global cooling
1.3: Weather is cold/snowing
1.4: Climate hasn't warmed/changed over the last (few) decade(s)
1.5: Oceans are cooling/not warming
1.6: Sea level rise is exaggerated/not accelerating
1.7: Extreme weather isn't increasing/has happened before/isn't linked to climate change
1.8: They changed the name from 'global warming' to 'climate change'
2.1: It's natural cycles/variation
2.2: It's non-greenhouse gas human climate forcings (aerosols, land use)
2.3: There's no evidence for greenhouse effect/carbon dioxide driving climate change
2.4: CO2 is not rising/ocean pH is not falling
2.5: Human CO2 emissions are miniscule/not raising atmospheric CO2
3.1: Climate sensitivity is low/negative feedbacks reduce warming
3.2: Species/plants/reefs aren't showing climate impacts yet/are benefiting from climate change
3.3: CO2 is beneficial/not a pollutant
3.4: It's only a few degrees (or less)
3.5: Climate change does not contribute to human conflict/threaten national security
3.6: Climate change doesn't negatively impact health
4.1: Climate policies (mitigation or adaptation) are harmful
4.2: Climate policies are ineffective/flawed
4.3: It's too hard to solve
4.4: Clean energy technology/biofuels won't work
4.5: People need energy (e.g., from fossil fuels/nuclear)
5.1: Climate-related science is uncertain/unsound/unreliable
5.2: Climate movement is alarmist/wrong/political/biased/hypocritical
5.3: Climate change (science or policy) is a conspiracy
0.0: None of the above"""

# Few-shot examples to guide the model
EXAMPLES = """
Example 1:
Text: "The Arctic ice is actually increasing in some areas, showing that global warming isn't happening as claimed."
Analysis:
1. Main claim: Arctic ice is increasing
Evidence: Mentions some areas of ice increase
2. This matches code 1.1 because it directly claims that ice cover isn't melting, using selective evidence about some areas.
Code: 1.1
Reason: Text specifically challenges ice melting narratives by claiming Arctic ice increase.

Example 2:
Text: "These climate scientists keep changing their story. First it was global warming, now it's climate change. They can't make up their minds!"
Analysis:
1. Main claim: Scientists changed terminology suspiciously
Evidence: Points to change from "global warming" to "climate change"
2. This matches code 1.8 because it explicitly mentions the naming change as evidence of inconsistency.
Code: 1.8
Reason: Text directly references the terminology change from global warming to climate change.

Example 3:
Text: "The Earth has natural cycles of warming and cooling that have been happening for millions of years. What we're seeing now is just part of that."
Analysis:
1. Main claim: Current climate change is natural
Evidence: References historical climate cycles
2. This matches code 2.1 because it attributes changes to natural variation.
Code: 2.1
Reason: Text explicitly argues that current changes are part of natural cycles."""

def prepare_prompt(text):
    system_prompt = f"""You are an expert analyst classifying climate change-related claims. Your task is to analyze the text and match it to the most appropriate category from the codebook.

Follow these steps:
1. Identify the main claim and supporting evidence in the text
2. Match it to the most specific applicable code
3. Provide clear reasoning for your choice

Codebook:
{CODEBOOK}

Here are examples of correct analyses:
{EXAMPLES}

Now analyze this text:
---
{text}
---

Format your response exactly like this:
Analysis:
1. Main claim and evidence:
[Describe main claim and evidence]

2. Based on this analysis:
Code: [number]
Reason: [Detailed explanation why this code fits best]"""
    return system_prompt

def classify_text(text, retry_count=3, retry_delay=1):
    """Classify climate-related text using local Ollama instance."""
    for attempt in range(retry_count):
        try:
            response = ollama.generate(
                model='llama3.2:latest',
                prompt=prepare_prompt(text),
                stream=False,
                options={
                    'temperature': 0.1,  # Lower temperature for more consistent outputs
                    'top_p': 0.9,
                    'num_predict': 500,  # Increased for more detailed analysis
                    'stop': ["\n\nText:", "Example"]  # Stop tokens
                }
            )
            
            output = response['response']
            
            if not output:
                raise ValueError("Empty response from model")
            
            print("\nRaw model output:", output)  # Debug output
                
            # Extract code using patterns
            code_patterns = [
                r'Code:\s*([0-9]+\.[0-9]+|0\.0)(?=\s|$)',  # Strict pattern for climate codes
                r'\b([0-9]+\.[0-9]+|0\.0)\b'  # Fallback pattern
            ]
            
            code = None
            for pattern in code_patterns:
                match = re.search(pattern, output, re.IGNORECASE)
                if match:
                    code = match.group(1).strip()
                    # Validate code against codebook
                    if code in [code.strip() for code in CODEBOOK.split('\n') if ':' in code]:
                        break
            
            if not code:
                # Keyword matching fallback
                text_lower = text.lower()
                keywords = {
                    '1.1': ['ice', 'permafrost', 'snow', 'melting'],
                    '1.2': ['ice age', 'cooling', 'cold'],
                    '1.3': ['weather', 'cold', 'snow'],
                    '1.4': ['warming', 'change', 'decade'],
                    '2.1': ['natural', 'cycle', 'variation'],
                    '5.1': ['uncertain', 'unreliable', 'science'],
                    '5.2': ['alarmist', 'political', 'biased']
                }
                
                scores = {code: sum(1 for word in words if word in text_lower)
                         for code, words in keywords.items()}
                
                if max(scores.values()) > 0:
                    code = max(scores.items(), key=lambda x: x[1])[0]
                else:
                    code = "0.0"
            
            # Extract reasoning
            reasoning_pattern = r'Reason:\s*(.*?)(?=(?:Analysis:|$))'
            reasoning_match = re.search(reasoning_pattern, output, re.IGNORECASE | re.DOTALL)
            reasoning = reasoning_match.group(1).strip() if reasoning_match else output.strip()
            
            return code, reasoning
                
        except Exception as e:
            if attempt == retry_count - 1:
                return "0.0", f"Processing failed after {retry_count} attempts: {str(e)}"
            time.sleep(retry_delay * (attempt + 1))
            continue

def process_dataframe(df, batch_size=50, save_interval=200):
    """Process the dataframe with progress tracking and periodic saving."""
    df_processed = df.copy()
    
    # Initialize columns for results
    if 'model_prediction' not in df_processed.columns:
        df_processed['model_prediction'] = None
    if 'reasoning' not in df_processed.columns:
        df_processed['reasoning'] = None
    
    unprocessed_mask = df_processed['model_prediction'].isna()
    total_unprocessed = unprocessed_mask.sum()
    
    if total_unprocessed == 0:
        print("All rows have already been processed!")
        return df_processed
    
    print(f"Processing {total_unprocessed} unprocessed rows...")
    
    with tqdm(total=total_unprocessed) as pbar:
        processed_count = 0
        
        for idx in df_processed[unprocessed_mask].index:
            try:
                text = str(df_processed.loc[idx, 'text'])
                
                # Skip invalid text
                if text.lower() in ['nan', 'none', ''] or len(text.strip()) < 10:
                    df_processed.loc[idx, 'model_prediction'] = "0.0"
                    df_processed.loc[idx, 'reasoning'] = "Text too short or empty"
                    processed_count += 1
                    pbar.update(1)
                    continue
                
                code, reasoning = classify_text(text)
                
                df_processed.loc[idx, 'model_prediction'] = code
                df_processed.loc[idx, 'reasoning'] = reasoning
                
                processed_count += 1
                pbar.update(1)
                
                # Save intermediate results
                if processed_count % save_interval == 0:
                    print(f"\nSaving intermediate results after processing {processed_count} rows...")
                    df_processed.to_csv('ollama_climate_results.csv', index=False)
                
                # Add delay to prevent overwhelming the model
                if processed_count % batch_size == 0:
                    time.sleep(1)
                    
            except Exception as e:
                print(f"\nError processing row {idx}: {str(e)}")
                continue
    
    # Print summary
    success_count = df_processed['model_prediction'].notna().sum() - df_processed['model_prediction'].eq('Error').sum()
    error_count = df_processed['model_prediction'].eq('Error').sum()
    print(f"\nProcessing complete!")
    print(f"Successfully processed: {success_count} rows")
    print(f"Errors: {error_count} rows")
    
    return df_processed

# Example usage:
if __name__ == "__main__":
    # Load your DataFrame here
    df = pd.read_csv('Data/contrarian_claims_reasons.csv')
    df_results = process_dataframe(df)
    #df_results.to_csv('ollama_climate_results.csv', index=False)

Processing 100 unprocessed rows...


  1%|          | 1/100 [00:14<23:12, 14.07s/it]


Raw model output: Analysis:
1. Main claim and evidence:
The text does not explicitly make a climate-related claim, but rather discusses the Food and Drug Administration's (FDA) request for input on online advertising of drugs and medical products. There is no mention of climate change or its impacts.

2. Based on this analysis:
Code: 0.0
Reason: The text does not contain any claims about climate change, ice cover, warming, cooling, sea level rise, greenhouse gases, or any other aspect of climate change. It appears to be unrelated to the topic and is simply a statement about an FDA request for input on online advertising.


  2%|▏         | 2/100 [00:21<16:39, 10.19s/it]


Raw model output: Analysis:
1. Main claim and evidence:
The text suggests that the globe hasn't warmed significantly since the late 19th century, citing factors such as data set reliability, proxy data selection, urbanization's impact on local weather stations.

2. Based on this analysis:
Code: 1.4
Reason: The text implies that climate change is not happening or has been minimal over the last few decades, which directly contradicts the widely accepted scientific consensus that the Earth has warmed since the late 19th century. This claim is supported by the discussion of data set reliability and proxy data selection, which could be interpreted as an attempt to cast doubt on the accuracy of temperature records.


...


Raw model output: Analysis:
1. Main claim and evidence:
The text attributes various historical events (First World War, Napoleon's campaigns) to natural climate variability or unrelated causes (fight at the local pub), suggesting that current climate change is not unusual.

2. Based on this analysis:
Code: 2.1
Reason: The text explicitly argues that current changes are part of natural cycles, referencing historical climate fluctuations and downplaying the significance of recent events like the cold snap. This matches code 2.1 because it attributes changes to natural variation, rather than attributing them to human activities or greenhouse gas emissions.


100%|██████████| 100/100 [12:33<00:00,  7.07s/it]


Raw model output: Analysis:
1. Main claim and evidence:
The text claims that climate models used by the IPCC may lead to overly confident predictions due to confirmation bias, as they share similar computer code and limitations. The evidence provided is a paper published in the Journal of Climate and empirical observations showing less warming than predicted by the IPCC models.

2. Based on this analysis:
Code: 5.1
Reason: This code fits best because the text challenges the reliability and accuracy of climate science, specifically questioning the confidence placed in climate model predictions. The main claim is that climate models are prone to confirmation bias, which undermines the trustworthiness of their results.


100%|██████████| 100/100 [12:34<00:00,  7.55s/it]


Processing complete!
Successfully processed: 100 rows
Errors: 0 rows


In [3]:
from sklearn.metrics import accuracy_score, cohen_kappa_score, confusion_matrix

def calculate_similarity_metrics(df_results, ground_truth_col, prediction_col):
    """
    Calculate various similarity metrics between two columns
    """
    # Remove rows where either column has missing values or errors
    valid_rows = df_results[[ground_truth_col, prediction_col]].dropna()
    valid_rows = valid_rows[valid_rows[prediction_col] != 'Error']
    
    # Convert all values to strings for comparison
    valid_rows = valid_rows.astype(str)
    
    if len(valid_rows) == 0:
        return {
            'accuracy': 0,
            'kappa': 0,
            'matching_cases': 0,
            'total_cases': 0,
            'percentage_match': 0
        }
    
    # Calculate metrics
    accuracy = accuracy_score(valid_rows[ground_truth_col], valid_rows[prediction_col])
    kappa = cohen_kappa_score(valid_rows[ground_truth_col], valid_rows[prediction_col])
    matching_cases = (valid_rows[ground_truth_col] == valid_rows[prediction_col]).sum()
    total_cases = len(valid_rows)
    percentage_match = (matching_cases / total_cases) * 100
    
    return {
        'accuracy': accuracy,
        'kappa': kappa,
        'matching_cases': matching_cases,
        'total_cases': total_cases,
        'percentage_match': percentage_match
    }

def analyze_code_similarities(df_results):
    """
    Analyze similarities between model outputs and human categorizations
    """
    # Comparisons to make
    comparisons = [
        ('original_code', 'model_code'),
        ('original_code', 'model_prediction'),
        ('replicated_code', 'model_code'),
        ('replicated_code', 'model_prediction')
    ]
    
    # Calculate metrics for each comparison
    results_dict = {}
    for truth_col, pred_col in comparisons:
        results_dict[f"{truth_col}_vs_{pred_col}"] = calculate_similarity_metrics(
            df_results, truth_col, pred_col
        )
    
    # Create a summary dataframe
    summary_df = pd.DataFrame({
        'Comparison': [k.replace('_', ' ').title() for k in results_dict.keys()],
        'Accuracy': [v['accuracy'] for v in results_dict.values()],
        'Kappa Score': [v['kappa'] for v in results_dict.values()],
        'Matching Cases': [v['matching_cases'] for v in results_dict.values()],
        'Total Cases': [v['total_cases'] for v in results_dict.values()],
        'Match Percentage': [v['percentage_match'] for v in results_dict.values()]
    })
    
    # Format the percentage column
    summary_df['Match Percentage'] = summary_df['Match Percentage'].round(2).astype(str) + '%'
    
    # Create detailed confusion matrices
    confusion_matrices = {}
    for truth_col, pred_col in comparisons:
        valid_rows = df_results[[truth_col, pred_col]].dropna()
        valid_rows = valid_rows[valid_rows[pred_col] != 'Error']  # Fixed: pred_col instead of prediction_col
        valid_rows = valid_rows.astype(str)
        
        if len(valid_rows) > 0:
            matrix = confusion_matrix(
                valid_rows[truth_col], 
                valid_rows[pred_col],
                labels=sorted(valid_rows[truth_col].unique())
            )
            confusion_matrices[f"{truth_col}_vs_{pred_col}"] = matrix
    
    return summary_df, confusion_matrices

# Run the analysis
summary_df, confusion_matrices = analyze_code_similarities(df_results)

# Display the results
print("\nSimilarity Analysis Summary:")
print("=" * 100)
print(summary_df.to_string(index=False))
print("\nDetailed Analysis:")
print("=" * 100)

# Determine which model is more similar to human coding
original_code_comparison = summary_df[summary_df['Comparison'].str.contains('Original Code')]
replicated_code_comparison = summary_df[summary_df['Comparison'].str.contains('Replicated Code')]

print("\nComparison with Original Code:")
better_for_original = original_code_comparison.iloc[original_code_comparison['Accuracy'].argmax()]
print(f"Better model: {better_for_original['Comparison']}")
print(f"Accuracy: {better_for_original['Accuracy']:.2%}")
print(f"Kappa Score: {better_for_original['Kappa Score']:.3f}")

print("\nComparison with Replicated Code:")
better_for_replicated = replicated_code_comparison.iloc[replicated_code_comparison['Accuracy'].argmax()]
print(f"Better model: {better_for_replicated['Comparison']}")
print(f"Accuracy: {better_for_replicated['Accuracy']:.2%}")
print(f"Kappa Score: {better_for_replicated['Kappa Score']:.3f}")

# Print confusion matrices with labels
for name, matrix in confusion_matrices.items():
    print(f"\nConfusion Matrix for {name}:")
    truth_col = name.split('_vs_')[0]
    categories = sorted(df_results[truth_col].astype(str).unique())
    print("\nCategories:", categories)
    print(matrix)


Similarity Analysis Summary:
                         Comparison  Accuracy  Kappa Score  Matching Cases  Total Cases Match Percentage
        Original Code Vs Model Code      0.61     0.490928              61          100            61.0%
  Original Code Vs Model Prediction      0.33     0.243194              33          100            33.0%
      Replicated Code Vs Model Code      0.56     0.449312              56          100            56.0%
Replicated Code Vs Model Prediction      0.31     0.231540              31          100            31.0%

Detailed Analysis:

Comparison with Original Code:
Better model: Original Code Vs Model Code
Accuracy: 61.00%
Kappa Score: 0.491

Comparison with Replicated Code:
Better model: Replicated Code Vs Model Code
Accuracy: 56.00%
Kappa Score: 0.449

Confusion Matrix for original_code_vs_model_code:

Categories: ['0.0', '1.1', '1.2', '1.6', '1.7', '2.1', '2.3', '3.1', '3.3', '4.1', '4.2', '5.1', '5.2']
[[35  2  0  0  2  1  2  0  1  1  2  2  5]
 [ 

In [5]:
df_results.to_csv('results_csvs/climate_prompt_small_model.csv', index=False)
print("Results saved")

Results saved
